In [4]:
from collections import Counter
import numpy as np
vocab = Counter(data.lower().split())
word_to_index = {word: idx for idx, (word, _) in enumerate(vocab.most_common())}
index_to_word = {idx: word for word, idx in word_to_index.items()}


total_count = sum(vocab.values())
word_freq = {word: count / total_count for word, count in vocab.items()}
subsample_prob = {word: 1 - np.sqrt(1e-5 / word_freq[word]) for word in vocab}
tokens = [word for word in vocab if np.random.rand() > subsample_prob[word]]
print(len(tokens))

250801


In [ ]:
len(skip_grams)

68020822

: 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
class SkipGramDataset(Dataset):
    def __init__(self, context_pairs, vocab_size, num_negative_samples=5):
        self.context_pairs = context_pairs
        self.vocab_size = vocab_size
        self.num_negative_samples = num_negative_samples

    def __len__(self):
        return len(self.context_pairs)

    def __getitem__(self, idx):
        target, context = self.context_pairs[idx]
        negative_samples = np.random.choice(
            self.vocab_size, self.num_negative_samples, replace=False
        )
        return target, context, torch.LongTensor(negative_samples)


In [6]:
def data_preprocessing(file_path):
    with open(file_path,'r') as file:
        data=file.read()
        print("read data")
    return data

file_path = "C:/users/pantm/Downloads/text8/text8.txt"
data = data_preprocessing(file_path)

read data


In [ ]:


import random
import string
import torch
from collections import Counter
import numpy as np
import re

def replace_punctuation_with_tokens(corpus):
    token_map = {
        r'\.': ' <PERIOD> ',
        r',': ' <COMMA> ',
        r'\?': ' <QUESTION> ',
        r'!': ' <EXCLAMATION> ',
        r':': ' <COLON> ',
        r';': ' <SEMICOLON> '
    }
    for pattern, token in token_map.items():
        corpus = re.sub(pattern, token, corpus)
    return corpus

def preprocess_text(corpus, min_count=5):
    """Preprocess text corpus into vocabulary and word indices."""
    # Lowercase and replace punctuation with tokens
    corpus = replace_punctuation_with_tokens(corpus.lower())
    # Split words and remove remaining punctuation
    words = [word.strip(string.punctuation) for word in corpus.split() if word.strip(string.punctuation)]
    # Count word frequencies and filter based on min_count
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    vocab = {word: i for i, (word, count) in enumerate(sorted_vocab) if count >= min_count}
    
    # Add UNK token for unknown words
    vocab['UNK'] = len(vocab)
    
    idx_to_word = {i: word for word, i in vocab.items()}
    
    # Convert words to indices, replacing out-of-vocabulary words with 'UNK'
    word_indices = [vocab.get(word, vocab['UNK']) for word in words]
    
    return vocab, idx_to_word, word_indices

# Define the data variable before using it

vocab, idx_to_word, word_indices = preprocess_text(data, min_count=1)  # Using min_count=1 for small example
print(len(vocab))


read data
253855


## Subsampling
- how does subsampling helps ?

In [4]:
def subsample_words(int_words, threshold = 1e-5):
  word_counts = Counter(int_words)
  total_n_words = len(int_words)

  freq_ratios = {word: count/total_n_words for word, count in word_counts.items()}
  p_drop = {word: 1 - np.sqrt(threshold/freq_ratios[word]) for word in word_counts}

  return [word for word in int_words if random.random() < (1 - p_drop[word])]


train_words = subsample_words(word_indices)
print(len(word_indices))
print(len(train_words))
print(len(train_words)/len(word_indices))

17005207
4980457
0.2928783519071541


In [1]:
import numpy as np
from scipy.special import expit as sigmoid
from collections import Counter

class SkipGram:
    def __init__(self, input_dim, hidden_dim, vocab_size, neg_sample_size=5, lr=0.01):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.neg_sample_size = neg_sample_size  # Number of negative samples
        self.lr = lr  # Learning rate

        # Initialize weights
        self.W1 = np.random.randn(input_dim, hidden_dim) * 0.01
        self.W2 = np.random.randn(hidden_dim, vocab_size) * 0.01
        self.b1 = np.zeros(hidden_dim)
        self.b2 = np.zeros(vocab_size)

        # Adam parameters
        self.m_W1, self.v_W1 = np.zeros_like(self.W1), np.zeros_like(self.W1)
        self.m_W2, self.v_W2 = np.zeros_like(self.W2), np.zeros_like(self.W2)
        self.m_b1, self.v_b1 = np.zeros_like(self.b1), np.zeros_like(self.b1)
        self.m_b2, self.v_b2 = np.zeros_like(self.b2), np.zeros_like(self.b2)
        self.beta1, self.beta2, self.eps = 0.9, 0.999, 1e-8
        self.t = 0  # Time step for Adam
    
    def forward(self, input_vec):
        """Compute hidden activations and output scores."""
        f = np.tanh(np.dot(input_vec, self.W1) + self.b1)
        output = np.dot(f, self.W2) + self.b2
        return f, output
    
    def sample_negatives(self, vocab_size, exclude_indices, n_samples):
        """Sample negative words while avoiding positives."""
        neg_samples = []
        while len(neg_samples) < n_samples:
            sample = np.random.randint(0, vocab_size)
            if sample not in exclude_indices:
                neg_samples.append(sample)
        return np.array(neg_samples)
    
    def update_adam(self, grad, m, v, param):
        """Apply Adam optimization."""
        self.t += 1
        m = self.beta1 * m + (1 - self.beta1) * grad
        v = self.beta2 * v + (1 - self.beta2) * (grad ** 2)
        m_hat = m / (1 - self.beta1 ** self.t)
        v_hat = v / (1 - self.beta2 ** self.t)
        param -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        return param, m, v

    def train(self, input_tensor, target_indices, vocab_size, epochs=25):
        """Train the model using Adam optimizer."""
        for epoch in range(epochs):
            total_loss = 0.0
            for i, input_idx in enumerate(input_tensor):
                input_vec = np.zeros(self.input_dim)
                input_vec[input_idx] = 1.0  # One-hot encoding

                # Forward pass
                f, output = self.forward(input_vec)

                # Negative sampling
                neg_sample_indices = self.sample_negatives(vocab_size, [target_indices[i]], self.neg_sample_size)

                # Compute loss
                pos_score = sigmoid(output[target_indices[i]])
                neg_scores = sigmoid(output[neg_sample_indices])
                loss = -np.log(pos_score + 1e-9) - np.sum(np.log(1 - neg_scores + 1e-9))
                total_loss += loss

                # Compute gradients
                grad_W2 = np.zeros_like(self.W2)
                grad_b2 = np.zeros_like(self.b2)
                
                grad_W2[:, target_indices[i]] = (pos_score - 1) * f
                grad_b2[target_indices[i]] = (pos_score - 1)
                
                for j, neg_idx in enumerate(neg_sample_indices):
                    grad_W2[:, neg_idx] += neg_scores[j] * f
                    grad_b2[neg_idx] += neg_scores[j]
                
                grad_f = (pos_score - 1) * self.W2[:, target_indices[i]] + np.sum(neg_scores * self.W2[:, neg_sample_indices], axis=1)
                grad_W1 = np.outer(input_vec, (1 - f ** 2) * grad_f)
                grad_b1 = (1 - f ** 2) * grad_f
                
                # Apply Adam updates
                self.W1, self.m_W1, self.v_W1 = self.update_adam(grad_W1, self.m_W1, self.v_W1, self.W1)
                self.W2, self.m_W2, self.v_W2 = self.update_adam(grad_W2, self.m_W2, self.v_W2, self.W2)
                self.b1, self.m_b1, self.v_b1 = self.update_adam(grad_b1, self.m_b1, self.v_b1, self.b1)
                self.b2, self.m_b2, self.v_b2 = self.update_adam(grad_b2, self.m_b2, self.v_b2, self.b2)
            
            print(f'Epoch {epoch + 1}, Loss: {total_loss:.4f}')


In [ ]:
data = "The quick brown fox jumps over the lazy dog"
vocab, idx_to_word, word_indices = preprocess_text(data, min_count=5)
skip_grams = generate_skip_grams(word_indices, window_size=2)

In [ ]:
# Initialize and train the SkipGram model
model = SkipGram(input_dim=1000, hidden_dim=50, vocab_size=len(vocab))
model.train(input_tensor, target_indices, vocab_size=len(vocab))


def preprocess_text(corpus, min_count=5):
    """Preprocess text corpus into vocabulary and word indices."""
    words = corpus.lower().split()
    word_counts = Counter(words)
    vocab = {word: i for i, (word, count) in enumerate(word_counts.items()) if count >= min_count}
    vocab['UNK'] = len(vocab)
    idx_to_word = {i: word for word, i in vocab.items()}
    word_indices = [vocab.get(word, vocab['UNK']) for word in words]
    return vocab, idx_to_word, word_indices

def generate_skip_grams(word_indices, window_size=5):
    """Generate skip-gram pairs with context window."""
    skip_grams = []
    for i in range(len(word_indices)):
        target_word = word_indices[i]
        context_range = range(max(0, i - window_size), min(len(word_indices), i + window_size + 1))
        for j in context_range:
            if i != j:
                skip_grams.append((target_word, word_indices[j]))
    return skip_grams


data = "hellow wrold ahowa reah and what is happenign and what you are trying to do m implementing skip gram is pain in the ass."
# Generate random data
vocab_size = len(data)  # Vocabulary size
hidden_dim = 50  # Hidden layer size
skip_gram_pairs = 500  # Number of skip-gram pairs

# Generate random word indices for input and targets
input_tensor = np.random.randint(0, vocab_size, size=skip_gram_pairs)
target_indices = np.random.randint(0, vocab_size, size=skip_gram_pairs)

# Initialize and train the SkipGram model
model = SkipGram(input_dim=vocab_size, hidden_dim=hidden_dim, vocab_size=vocab_size)
model.train(input_tensor, target_indices, vocab_size)

In [ ]:
def generate_skip_grams(word_indices, window_size=5):
    """Generate skip-gram pairs with context window"""
    skip_grams = []
    for i in range(len(word_indices)):
        target_word = word_indices[i]
        # Get context words within window
        context_range = range(max(0, i - window_size), min(len(word_indices), i + window_size + 1))
        for j in context_range:
            if i != j:  # Skip the target word itself
                skip_grams.append((target_word, word_indices[j]))
    return skip_grams

skip_grams = generate_skip_grams(word_indices, window_size=2)

def generate_negative_samples(target_word, context_word, vocab_size, num_samples):
    """Generate negative samples that are different from the target-context pair"""
    negative_samples = []
    while len(negative_samples) < num_samples:
        # Generate a random context word
        neg_context = np.random.randint(0, vocab_size)
        # Make sure it's not the same as the positive context
        if neg_context != context_word and (target_word, neg_context) not in negative_samples:
            negative_samples.append((target_word, neg_context))
    return negative_samples

# Generate negative samples
num_negative_samples = 5
negTrainSet = []

for target_word, context_word in skip_grams:
    # Generate negative samples for this target word
    neg_samples = generate_negative_samples(target_word, context_word, len(vocab), num_negative_samples)
    negTrainSet.extend(neg_samples)

print(f"Number of positive examples: {len(skip_grams)}")
print(f"Number of negative examples: {len(negTrainSet)}")
# # Convert to tensors for model input
# input_tensor = torch.tensor([pair[0] for pair in skip_grams], dtype=torch.long)
# target_indices = torch.tensor([pair[1] for pair in skip_grams], dtype=torch.long)
#print(data)

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from collections import Counter

class SkipGram:
    def __init__(self, vocab_size, embedding_dim, num_negative_samples=5):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.num_negative_samples = num_negative_samples
        
        # Initialize embeddings
        self.W1 = torch.randn(vocab_size, embedding_dim) * 0.1
        self.W2 = torch.randn(vocab_size, embedding_dim) * 0.1
        
        self.eta = 0.01
        
    def forward(self, target_word, context_word, negative_samples):
        # Get embeddings
        target_emb = self.W1[target_word]  # [batch_size, embedding_dim]
        context_emb = self.W2[context_word]  # [batch_size, embedding_dim]
        neg_emb = self.W2[negative_samples]  # [batch_size, num_neg_samples, embedding_dim]
        
        # Positive score
        pos_score = torch.sum(target_emb * context_emb, dim=1)
        pos_loss = -F.logsigmoid(pos_score)
        
        # Negative score
        neg_score = torch.bmm(neg_emb, target_emb.unsqueeze(2)).squeeze()
        neg_loss = -torch.sum(F.logsigmoid(-neg_score), dim=1)
        
        return pos_loss + neg_loss, target_emb, context_emb, neg_emb

    def gradient_descent(self, dataloader, n_epoch=10):
        for epoch in range(n_epoch):
            total_loss = 0
            for target_word, context_word, negative_samples in dataloader:
                loss, target_emb, context_emb, neg_emb = self.forward(target_word, context_word, negative_samples)
                total_loss += loss.mean().item()

                # Gradients for positive samples
                d_pos = -torch.sigmoid(-torch.sum(target_emb * context_emb, dim=1)).unsqueeze(1) * context_emb
                d_context = -torch.sigmoid(-torch.sum(target_emb * context_emb, dim=1)).unsqueeze(1) * target_emb

                # Gradients for negative samples
                d_neg = torch.sigmoid(torch.bmm(neg_emb, target_emb.unsqueeze(2)).squeeze()).unsqueeze(2) * neg_emb
                d_target_neg = torch.sum(torch.sigmoid(torch.bmm(neg_emb, target_emb.unsqueeze(2)).squeeze()).unsqueeze(2) * neg_emb, dim=1)

                # Update embeddings
                self.W1[target_word] -= self.eta * (d_pos + d_target_neg)
                self.W2[context_word] -= self.eta * d_context
                self.W2[negative_samples] -= self.eta * d_neg

            print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}')

    def get_word_embedding(self, word_idx):
        return self.W1[word_idx].detach().numpy()

class SkipGramDataset(torch.utils.data.Dataset):
    def __init__(self, text, window_size=2, num_negative_samples=5):
        words = text.lower().split()
        word_counts = Counter(words)
        self.vocab = {word: idx for idx, (word, _) in enumerate(word_counts.most_common())}
        self.vocab_size = len(self.vocab)
        self.idx_to_word = {idx: word for word, idx in self.vocab.items()}
        
        word_indices = [self.vocab[word] for word in words if word in self.vocab]
        
        self.skip_grams = []
        for i in range(len(word_indices)):
            for j in range(max(0, i - window_size), min(len(word_indices), i + window_size + 1)):
                if i != j:
                    self.skip_grams.append((word_indices[i], word_indices[j]))
        
        self.num_negative_samples = num_negative_samples
        
        word_freqs = np.array([count for _, count in word_counts.most_common()])
        word_freqs = word_freqs ** 0.75
        self.sampling_weights = word_freqs / np.sum(word_freqs)

    def __len__(self):
        return len(self.skip_grams)

    def __getitem__(self, idx):
        target_word, context_word = self.skip_grams[idx]
        negative_samples = np.random.choice(
            self.vocab_size, 
            size=self.num_negative_samples, 
            p=self.sampling_weights, 
            replace=False
        )
        return torch.tensor(target_word), torch.tensor(context_word), torch.tensor(negative_samples)

def train_skip_gram_with_negative_sampling(text, embedding_dim=100, window_size=2, 
                                          num_negative_samples=5, batch_size=32, epochs=5):
    dataset = SkipGramDataset(text, window_size, num_negative_samples)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = SkipGram(dataset.vocab_size, embedding_dim, num_negative_samples)
    model.gradient_descent(dataloader, n_epoch=epochs)
    
    return model, dataset.vocab, dataset.idx_to_word

def find_similar_words(word, model, vocab, idx_to_word, top_k=5):
    if word not in vocab:
        return []
    
    word_idx = vocab[word]
    word_vector = model.get_word_embedding(word_idx)
    
    similarities = []
    for idx in range(len(vocab)):
        if idx != word_idx:
            vector = model.get_word_embedding(idx)
            similarity = np.dot(word_vector, vector) / (np.linalg.norm(word_vector) * np.linalg.norm(vector))
            similarities.append((idx_to_word[idx], similarity))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

# Example usage
if __name__ == "__main__":
    #text = "The wide road shimmered in the hot sun. The algorithm was developed to produce software."
    model, vocab, idx_to_word = train_skip_gram_with_negative_sampling(
        data, embedding_dim=50, epochs=10
    )
    
    similar_words = find_similar_words("the", model, vocab, idx_to_word)
    print(f"Words similar to 'the': {similar_words}")